In [2]:
import pandas as pd
import numpy as np

import scipy.stats

from statsmodels.stats.weightstats import _tconfint_generic

### Вопрос 2

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Будьте осторожны при использовании метода std()! Нас интересует только несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на n-1.

In [2]:
df = pd.read_csv('water.txt', sep='\t', error_bad_lines=False)

In [3]:
df.head(10)

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15
7,South,Bournemouth,1299,78
8,North,Bradford,1637,10
9,South,Brighton,1359,84


In [4]:
df.mean()

mortality    1524.147541
hardness       47.180328
dtype: float64

In [5]:
df.std(ddof=1)

mortality    187.668754
hardness      38.093966
dtype: float64

### Доверительный интервал с заданным интервалом доверия - вариант №1

In [6]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [7]:
mean_confidence_interval(df['mortality'])

(1524.1475409836066, 1476.0833413552848, 1572.2117406119285)

### Доверительный интервал с заданным интервалом доверия - вариант №2

In [10]:
_tconfint_generic(df['mortality'].mean(), 
                  df['mortality'].std(ddof=1) / np.sqrt(len(df)), len(df) - 1, 0.05, 'two-sided')

(1476.0833413552848, 1572.2117406119285)

### Вопрос 3

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [12]:
df_south = df[df['location'] == 'South'].copy()
df_south.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
2,South,Birmingham,1466,5
7,South,Bournemouth,1299,78
9,South,Brighton,1359,84
10,South,Bristol,1392,73


In [13]:
mean_confidence_interval(df_south['mortality'])

(1376.8076923076924, 1320.1517462936238, 1433.463638321761)

In [28]:
# _tconfint_generic(df_south['mortality'].mean(), 
#                   df_south['mortality'].std(ddof=1) / np.sqrt(len(df_south)), len(df_south) - 1, 0.05, 'two-sided')

### Вопрос 4

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

In [30]:
df_north = df[df['location'] == 'North'].copy()
df_north.head()

,location,town,mortality,hardness
1,North,Birkenhead,1668,17
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15


In [77]:
mean_confidence_interval(df_north['mortality'])

(1633.6, 1586.5605251961385, 1680.6394748038613)

In [32]:
# _tconfint_generic(df_north['mortality'].mean(), 
#                   df_north['mortality'].std(ddof=1) / np.sqrt(len(df_north)), len(df_north) - 1, 0.05, 'two-sided')

### Вопрос 5

Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [24]:
# df_south.iloc[:2]
df_south['hardness'].head()

0     105
2       5
7      78
9      84
10     73
Name: hardness, dtype: int64

In [80]:
mean_confidence_interval(df_south['hardness'])

(69.76923076923077, 53.467198692036106, 86.07126284642544)

In [84]:
df_north['hardness'].head()

1    17
3    14
4    18
5    10
6    15
Name: hardness, dtype: int64

In [85]:
mean_confidence_interval(df_north['hardness'])

(30.4, 21.42248728572426, 39.37751271427574)

In [33]:
# _tconfint_generic(df[df.location == 'South'].hardness.mean(), 
#                   df[df.location == 'South'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'South'])), 
#                   len(df[df.location == 'South']) - 1, 0.05, 'two-sided')

In [34]:
# _tconfint_generic(df[df.location == 'North'].hardness.mean(), 
#                   df[df.location == 'North'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'North'])), 
#                   len(df[df.location == 'North']) - 1, 0.05, 'two-sided')

### Вопрос 6

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью $\pm$0.1?

In [3]:
np.ceil((scipy.stats.norm.ppf(1-0.05/2) / 0.1)**2)

385.0

In [5]:
round(scipy.stats.norm.ppf(1-0.05/2), 2)

1.96

In [12]:
np.ceil(((scipy.stats.norm.ppf(1-0.05/2))**2 * 1**2) / 0.1**2)

385.0